# Error Handling and Debugging in Flask

> **Teaching Note**: This guide covers essential techniques for debugging Flask applications during development and implementing user-friendly, secure error handling in production.

---

## 1. Understanding Flask's Built-in Debugger

Flask includes a powerful interactive debugger that helps during development.

### Enabling the Debugger

In [ ]:
from flask import Flask

app = Flask(__name__)
app.debug = True  # Enable debug mode

# Or set environment variable
# export FLASK_ENV=development
# export FLASK_DEBUG=1

if __name__ == '__main__':
    app.run(debug=True)

### Debugger Features
- **Interactive traceback**: Click on stack frames to inspect variables
- **Console access**: Execute Python code in the context of any stack frame
- **Auto-reload**: Automatically restarts when code changes

> **⚠️ Security Warning**: Never enable debug mode in production! It can expose sensitive information and allow arbitrary code execution.

---

## 2. Common Error Types in Flask

### HTTP Errors
- **404 Not Found**: Requested resource doesn't exist
- **403 Forbidden**: Access denied
- **400 Bad Request**: Invalid request data
- **500 Internal Server Error**: Server-side error

### Python Exceptions
- `ValueError`, `TypeError`, `KeyError`, etc.
- Database connection errors
- File I/O errors
- Import errors

---

## 3. Custom Error Pages

### Basic Error Handlers

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.errorhandler(404)
def not_found_error(error):
    return render_template('errors/404.html'), 404

@app.errorhandler(500)
def internal_error(error):
    # Log the error for debugging
    app.logger.error(f'Server Error: {error}')
    return render_template('errors/500.html'), 500

@app.errorhandler(403)
def forbidden_error(error):
    return render_template('errors/403.html'), 403

### Generic Exception Handler

In [ ]:
@app.errorhandler(Exception)
def handle_exception(e):
    # Log the exception
    app.logger.error(f'Unhandled exception: {str(e)}')
    
    # Return appropriate response based on request type
    if request.is_json:
        return {'error': 'Internal server error'}, 500
    else:
        return render_template('errors/500.html'), 500

### Error Templates

**templates/errors/404.html**

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Page Not Found</title>
</head>
<body>
    <h1>404 - Page Not Found</h1>
    <p>The page you're looking for doesn't exist.</p>
    <a href="{{ url_for('index') }}">Go Home</a>
</body>
</html>

**templates/errors/500.html**

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Server Error</title>
</head>
<body>
    <h1>500 - Internal Server Error</h1>
    <p>Something went wrong on our end. We're working to fix it!</p>
    <a href="{{ url_for('index') }}">Go Home</a>
</body>
</html>

---

## 4. Advanced Error Handling

### Custom Error Classes

In [ ]:
class ValidationError(Exception):
    """Custom validation error"""
    def __init__(self, message, status_code=400):
        super().__init__(message)
        self.message = message
        self.status_code = status_code

class AuthenticationError(Exception):
    """Custom authentication error"""
    def __init__(self, message="Unauthorized", status_code=401):
        super().__init__(message)
        self.message = message
        self.status_code = status_code

# Error handlers for custom exceptions
@app.errorhandler(ValidationError)
def handle_validation_error(error):
    return {'error': error.message}, error.status_code

@app.errorhandler(AuthenticationError)
def handle_auth_error(error):
    return {'error': error.message}, error.status_code

### Usage in Routes

In [ ]:
@app.route('/api/users', methods=['POST'])
def create_user():
    try:
        data = request.get_json()
        if not data or 'email' not in data:
            raise ValidationError('Email is required')
        
        # Process user creation
        user = create_user_in_db(data)
        return {'user': user.to_dict()}, 201
        
    except ValidationError as e:
        # This will be handled by the custom error handler
        raise e
    except Exception as e:
        app.logger.error(f'User creation failed: {str(e)}')
        raise  # This will be caught by the generic exception handler

---

## 5. Effective Debugging Techniques

### Logging Configuration

In [ ]:
import logging
from logging.handlers import RotatingFileHandler
import os

def setup_logging(app):
    if not app.debug and not app.testing:
        # Production logging
        if not os.path.exists('logs'):
            os.mkdir('logs')
        
        file_handler = RotatingFileHandler(
            'logs/app.log', 
            maxBytes=10240000,  # 10MB
            backupCount=10
        )
        file_handler.setFormatter(logging.Formatter(
            '%(asctime)s %(levelname)s: %(message)s [in %(pathname)s:%(lineno)d]'
        ))
        file_handler.setLevel(logging.INFO)
        app.logger.addHandler(file_handler)
        app.logger.setLevel(logging.INFO)
        app.logger.info('Application startup')

# Initialize logging
setup_logging(app)

### Debugging with Breakpoints

In [ ]:
@app.route('/debug-route')
def debug_route():
    user_id = request.args.get('user_id')
    
    # Set breakpoint for debugging
    import pdb; pdb.set_trace()  # Traditional debugger
    
    # Or use Flask's built-in debugger
    # This will trigger the interactive debugger if debug=True
    if user_id is None:
        raise ValueError("User ID is required")
    
    return f"User ID: {user_id}"

### Using Flask-DebugToolbar

In [ ]:
# First install: pip install Flask-DebugToolbar

from flask_debugtoolbar import DebugToolbarExtension

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your-secret-key'
app.debug = True

# Initialize debug toolbar
toolbar = DebugToolbarExtension(app)

# The toolbar will automatically appear on all pages when debug=True

---

## 6. Production Error Handling

### Sentry Integration (Error Monitoring)

In [ ]:
# First install: pip install sentry-sdk[flask]

import sentry_sdk
from sentry_sdk.integrations.flask import FlaskIntegration

# Initialize Sentry
sentry_sdk.init(
    dsn="your-sentry-dsn",
    integrations=[FlaskIntegration()],
    traces_sample_rate=1.0
)

app = Flask(__name__)

### Health Check Endpoint

In [ ]:
@app.route('/health')
def health_check():
    try:
        # Check database connection
        db.session.execute('SELECT 1')
        return {'status': 'healthy', 'database': 'connected'}, 200
    except Exception as e:
        app.logger.error(f'Health check failed: {str(e)}')
        return {'status': 'unhealthy', 'error': str(e)}, 500

---

## 7. Best Practices

### 1. Environment-Specific Configuration

In [ ]:
class Config:
    DEBUG = False
    TESTING = False
    LOG_LEVEL = logging.INFO

class DevelopmentConfig(Config):
    DEBUG = True
    LOG_LEVEL = logging.DEBUG

class ProductionConfig(Config):
    LOG_LEVEL = logging.WARNING

# Load appropriate config
app.config.from_object(ProductionConfig if os.environ.get('FLASK_ENV') == 'production' else DevelopmentConfig)

### 2. User-Friendly Error Messages

In [ ]:
@app.errorhandler(404)
def not_found_error(error):
    # Don't expose internal details to users
    return {
        'error': 'The requested resource was not found',
        'status_code': 404
    }, 404

### 3. Comprehensive Logging

In [ ]:
@app.errorhandler(Exception)
def handle_exception(e):
    # Log with context
    app.logger.error(
        f'Exception on {request.path} [{request.method}]: {str(e)}',
        extra={
            'user_id': getattr(g, 'user_id', 'anonymous'),
            'ip_address': request.remote_addr,
            'user_agent': request.user_agent.string
        }
    )
    return {'error': 'Internal server error'}, 500

### 4. Testing Error Handlers

In [ ]:
# First install: pip install pytest

import pytest
from your_app import app

def test_404_error():
    with app.test_client() as client:
        response = client.get('/non-existent-page')
        assert response.status_code == 404
        assert b'Page Not Found' in response.data

def test_500_error():
    # Simulate an error
    with app.test_client() as client:
        with app.app_context():
            # Force an exception
            response = client.get('/route-that-causes-error')
            assert response.status_code == 500

---

## Summary Checklist

✅ **Development**:
- Enable Flask debugger
- Use logging with appropriate levels
- Test error handlers

✅ **Production**:
- Disable debug mode
- Implement custom error pages
- Set up proper logging
- Use error monitoring (Sentry, etc.)
- Provide user-friendly error messages

✅ **Security**:
- Never expose stack traces to users
- Log errors securely
- Validate all inputs
- Handle exceptions gracefully

> **Teaching Tip**: Emphasize the difference between development and production environments. Students should understand that good error handling is crucial for both debugging during development and providing a good user experience in production.